In [5]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import scipy.sparse.linalg
from scipy.sparse import csr_matrix, lil_matrix

# Problema: A, T y W

In [6]:
T = np.zeros((15,3))
T[12,0] = 1
T[13,1] = 1
T[14,2] = 1
W = np.zeros((15,21))
for i in range(0,9):
    W[i,i] = 1
    W[i,i+9] = 1
    W[9+i%3,i] = 1
W[14,20]= -1
W[13,19]= -1
W[12,18] = -1

#plt.imshow(T, cmap='Greens')
#plt.gca().axes.get_yaxis().set_visible(False)
#plt.gca().axes.get_xaxis().set_visible(False)
#plt.imshow(W, cmap='PRGn')
#plt.gca().axes.get_yaxis().set_visible(False)
#plt.gca().axes.get_xaxis().set_visible(False)

## Gurobi

In [8]:
import gurobipy as gp
from gurobipy import Model, GRB, quicksum

In [9]:
model = gp.Model('Intento')
y = model.addVars([1,2], lb = 0, vtype = GRB.CONTINUOUS)
obj = np.array([-24,-28])
model.modelSense = GRB.MINIMIZE
model.setObjective( quicksum(y[a]*obj[a-1] for a in range(1,3)) )

lhs_ineq = np.array([[6,10],[8,5],[1,0],[0,1]])
rhs_ineq = np.array([2400,1600,500,100])

model.addMConstr (lhs_ineq, x = None, sense='<=', b = rhs_ineq, name="cons" )
model.optimize()
print("\nResultado:\n")
print(y,"\n")
print("Pi del modelo: \n")
print(model.pi)

Academic license - for non-commercial use only - expires 2021-06-26
Using license file C:\Users\melir\gurobi.lic
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 4 rows, 2 columns and 6 nonzeros
Model fingerprint: 0xb42b4eb3
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [2e+01, 3e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 2e+03]
Presolve removed 4 rows and 2 columns
Presolve time: 0.04s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -6.1000000e+03   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.05 seconds
Optimal objective -6.100000000e+03

Resultado:

{1: <gurobi.Var C0 (value 137.5)>, 2: <gurobi.Var C1 (value 100.0)>} 

Pi del modelo: 

[0.0, -3.0, 0.0, -13.0]


In [ ]:
def linProgram():
    

## Escenarios
Creamos 5000 escenarios para el vector $\zeta = (r_1,r_2,r_3,d_1,d_2,d_3)$ de forma uniforme. Pueden tomar valores enteros, $ 20 \leq r_1,r_2,r_3 \leq 30$ y $(d_1,d_2,d_3) \in \{(0,8,25),(1,15,35),(1,11,40),(3,20,55),(4,20,70)\}$. Tenemos entonces $10\times 10\times 10 \times 5 = 5000$ valores posibles para $\zeta$. Cada uno lo tomamos con probabilidad $p = \frac{1}{5000}$.